In [1]:
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point

# Function

In [19]:
def streetTalk(lon,lat,cityname,networkType='drive'):
    '''
    This function takes a lat/lng pair and a city name as inputs, and returns
    a conversational string stating nearest street as well as enclosing streets
    point: shapely.geometry.Point
    cityname: string
    networkType: string
    '''
    try:
        #If the map of city is downloaded, use them directly
        #citymap_node = gpd.read_file('data/'+cityname+'/nodes/nodes.shp')
        citymap_edge = gpd.read_file('data/'+cityname+'/edges/edges.shp')
        #print('exist!')
    except:
        #If the map is not downloaded, download the map of city via OpenStreetMap
        G = ox.graph_from_place(cityname + ', USA', network_type=networkType)
        #Save the map in shapefile locally
        ox.save_load.save_graph_shapefile(G, filename=cityname, folder=None, encoding='utf-8')
        print('Map of ' + cityname + 'is saved in shapefile locally!')
        #citymap_node = gpd.read_file('data/'+cityname+'/nodes/nodes.shp')
        citymap_edge = gpd.read_file('data/'+cityname+'/edges/edges.shp')
    
    point = Point(lon,lat)
    nearest_name, nearest_id, from_id, to_id = nearestSegment(point,citymap_edge)
    
    enclosing_from = intersectingStreets(from_id,citymap_edge)
    enclosing_to = intersectingStreets(to_id,citymap_edge)
    
    street_from  = enclosing_from[1] if enclosing_from[0] == nearest_name else enclosing_from[0]
    street_to  = enclosing_to[1] if enclosing_to[0] == nearest_name else enclosing_to[0]
    
    #print(len(citymap_node),len(citymap_edge))
    conversational = "{} between {} and {}".format(nearest_name,street_from,street_to)
    print(conversational)
    return conversational

In [20]:
def nearestSegment(point,city):
    '''
    This function takes a lat/lon point and a geodataframe of city shapefile
    as inputs and returns a name and unique id for nearest street segment to
    that point as well as unique ids for the segment's enclosing intersections
    point: shapely.geometry.Point (lon/lat)
    city: geodataframe of edges of segments in city
    return: nearest_name, nearest_id, from_id, to_id
    '''
    idx = city.geometry.distance(point).sort_values().index[0]
    nearest_name = city.loc[idx,'name']
    nearest_id = city.loc[idx,'osmid']
    from_id = city.loc[idx,'from']
    to_id = city.loc[idx,'to']
    return nearest_name, nearest_id, from_id, to_id

In [21]:
def intersectingStreets(node_id,city):
    '''
    This function takes a lat/lng pair as an input, and returns the names of
    intersecting streets
    node_id: node id, string
    city: geodataframe of edges of segments in city
    return: street_name, street_name_1
    '''
    return city[(city['from'] == node_id) | (city['to'] == node_id)].name.unique()
    

# Test

In [1]:
#Import test data
import pandas as pd
data = pd.read_csv('data/test_data/squid_test_data.csv')

In [2]:
data.head()

,id,timestamp,v_value,geometry,image_url,trip_id,original_x,original_y,snapped_x,snapped_y
0,1,44:56.1,0.47553,0101000020E610000021882911DE8B5DC092A1BF28000C...,http://storage6.openstreetcam.org/files/photo/...,973407,-118.185476,34.093767,-118.185429,34.093755
1,2,45:01.1,0.78636,0101000020E6100000B062E053DF8B5DC0965A1099FA0B...,http://storage6.openstreetcam.org/files/photo/...,973407,-118.185551,34.093602,-118.185506,34.093585
2,3,45:03.1,1.30535,0101000020E6100000DB68A169E08B5DC059149BE7F60B...,http://storage6.openstreetcam.org/files/photo/...,973407,-118.185634,34.093500,-118.185572,34.093472
3,4,45:05.2,1.29946,0101000020E61000003F0D51EEE18B5DC0C16130A7F20B...,http://storage6.openstreetcam.org/files/photo/...,973407,-118.185728,34.093376,-118.185665,34.093343
4,5,45:07.1,1.13484,0101000020E610000057954B7EE38B5DC042836005EF0B...,http://storage6.openstreetcam.org/files/photo/...,973407,-118.185834,34.093277,-118.185760,34.093232


In [3]:
data['conversational string'] = ""
data.head()

,id,timestamp,v_value,geometry,image_url,trip_id,original_x,original_y,snapped_x,snapped_y,conversational string
0,1,44:56.1,0.47553,0101000020E610000021882911DE8B5DC092A1BF28000C...,http://storage6.openstreetcam.org/files/photo/...,973407,-118.185476,34.093767,-118.185429,34.093755,
1,2,45:01.1,0.78636,0101000020E6100000B062E053DF8B5DC0965A1099FA0B...,http://storage6.openstreetcam.org/files/photo/...,973407,-118.185551,34.093602,-118.185506,34.093585,
2,3,45:03.1,1.30535,0101000020E6100000DB68A169E08B5DC059149BE7F60B...,http://storage6.openstreetcam.org/files/photo/...,973407,-118.185634,34.093500,-118.185572,34.093472,
3,4,45:05.2,1.29946,0101000020E61000003F0D51EEE18B5DC0C16130A7F20B...,http://storage6.openstreetcam.org/files/photo/...,973407,-118.185728,34.093376,-118.185665,34.093343,
4,5,45:07.1,1.13484,0101000020E610000057954B7EE38B5DC042836005EF0B...,http://storage6.openstreetcam.org/files/photo/...,973407,-118.185834,34.093277,-118.185760,34.093232,


In [13]:
data.keys()

Index(['id', 'timestamp', 'v_value', 'geometry', 'image_url', 'trip_id',
       'original_x', 'original_y', 'snapped_x', 'snapped_y',
       'conversational string'],
      dtype='object')

In [25]:
for i in range(len(data)):
    data.iloc[i,10] = streetTalk(data.iloc[i,6],data.iloc[i,7],'Los Angeles')

Via Marisol between Vallejo Villas Street and Via Mia
Via Marisol between Vallejo Villas Street and Via Mia
Via Marisol between Vallejo Villas Street and Via Mia
Via Marisol between Vallejo Villas Street and Via Mia
Via Marisol between Vallejo Villas Street and Via Mia
Via Marisol between Vallejo Villas Street and Via Mia
Via Marisol between Via Mia and Monterey Road
Via Marisol between Via Mia and Monterey Road
Via Marisol between Via Mia and Monterey Road
Via Marisol between Via Mia and Monterey Road
Via Marisol between Via Mia and Monterey Road
Via Marisol between Via Mia and Monterey Road
Via Marisol between Via Mia and Monterey Road
Via Marisol between Via Mia and Monterey Road
Via Marisol between Via Mia and Monterey Road
Via Marisol between Via Mia and Monterey Road
Via Marisol between Via Mia and Monterey Road
Via Marisol between Via Mia and Monterey Road
Via Marisol between Via Mia and Monterey Road
Via Marisol between Via Mia and Monterey Road
Via Marisol between Via Mia and 

IndexError: index 1 is out of bounds for axis 0 with size 1

In [29]:
data[data['conversational string']==""]

,id,timestamp,v_value,geometry,image_url,trip_id,original_x,original_y,snapped_x,snapped_y,conversational string
63,64,47:30.2,1.22950,0101000020E61000003F78F596138C5DC011F2F92A640B...,http://storage6.openstreetcam.org/files/photo/...,973407,-118.188637,34.088978,-118.188696,34.088994,
64,65,47:32.2,1.40779,0101000020E6100000B787A8B9138C5DC096432B84630B...,http://storage6.openstreetcam.org/files/photo/...,973407,-118.188475,34.088864,-118.188704,34.088974,
65,66,47:34.1,1.35059,0101000020E6100000EE4377D80C8C5DC0A21A129B5F0B...,http://storage6.openstreetcam.org/files/photo/...,973407,-118.188301,34.088813,-118.188284,34.088855,
66,67,47:36.2,0.82016,0101000020E61000007CAC64E1098C5DC0290199EB5D0B...,http://storage6.openstreetcam.org/files/photo/...,973407,-118.188122,34.088758,-118.188103,34.088804,
67,68,47:38.2,0.73556,0101000020E6100000686E8DAE068C5DC02F9D261A5C0B...,http://storage6.openstreetcam.org/files/photo/...,973407,-118.187925,34.088708,-118.187908,34.088748,
68,69,47:40.1,1.45878,0101000020E610000084F1CA83038C5DC02A0E6F64590B...,http://storage6.openstreetcam.org/files/photo/...,973407,-118.187757,34.088622,-118.187714,34.088666,
69,70,47:42.1,0.94679,0101000020E610000060B2F040008C5DC01891BB72580B...,http://storage6.openstreetcam.org/files/photo/...,973407,-118.187670,34.088479,-118.187516,34.088637,
70,71,47:44.1,0.64751,0101000020E610000060B2F040008C5DC01891BB72580B...,http://storage6.openstreetcam.org/files/photo/...,973407,-118.187639,34.088324,-118.187516,34.088637,
71,72,47:46.1,0.64191,0101000020E6100000BB31E848038C5DC05006C488490B...,http://storage6.openstreetcam.org/files/photo/...,973407,-118.187626,34.088161,-118.187700,34.088182,
72,73,47:48.2,0.82615,0101000020E610000047E3323C048C5DC09D4D3DD1440B...,http://storage6.openstreetcam.org/files/photo/...,973407,-118.187621,34.088000,-118.187759,34.088038,


In [30]:
streetTalk(-118.188637,34.088978,'Los Angeles')

IndexError: index 1 is out of bounds for axis 0 with size 1